In [1]:
# # split the pdf into separate pages

# import PyPDF2
# from src.document_generation import setup_logger
# import logging

# logger = logging.getLogger('logger_name')
# if logger.hasHandlers():
#     logger.handlers.clear()  # Clear existing handlers to avoid duplicates
# logger.setLevel(logging.DEBUG)
# handler = logging.StreamHandler()
# formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
# handler.setFormatter(formatter)
# logger.addHandler(handler)
# logger.propagate = False

# # Input PDF file path
# input_pdf_path = "../input_data/Der Weltkrieg v7 East Front.pdf"
# output_folder = "../input_data/Der Weltkrieg v7"

# # Ensure the output folder exists
# os.makedirs(output_folder, exist_ok=True)

# # Open the PDF and split pages
# try:
#     with open(input_pdf_path, 'rb') as file:
#         pdf_reader = PyPDF2.PdfReader(file)
#         num_pages = len(pdf_reader.pages)
#         logger.info(f"Extracted {num_pages} pages from PDF")

#         for i, page in enumerate(pdf_reader.pages):
#             # Create a new PDF writer for each page
#             pdf_writer = PyPDF2.PdfWriter()
#             pdf_writer.add_page(page)

#             # Save the current page to a new file
#             output_file_path = os.path.join(output_folder, f"page_{i+1:03d}.pdf")
#             with open(output_file_path, 'wb') as output_file:
#                 pdf_writer.write(output_file)
            
#             logger.debug(f"Saved page {i+1} to {output_file_path}")
#             if i == 10:
#                 break

#     logger.info(f"All pages have been split and saved to {output_folder}")

# except Exception as e:
#     logger.error(f"An error occurred: {e}")


In [2]:
%%time
%load_ext autoreload
%autoreload 2

# Standard Python modules
import time 
import numpy as np
import os
import sys
import json
import re
import glob
import asyncio
import aiohttp
import openai
import logging
from typing import Dict, Tuple, List, Callable
from dotenv import load_dotenv
from pathlib import Path

# Decorator to log wall time
def log_execution_time(func: Callable):
    async def wrapper(*args, **kwargs):
        start = time.time()
        result = await func(*args, **kwargs)
        logger.info(f"Finished {func.__name__} in {time.time() - start:.2f} seconds.")
        return result
    return wrapper

def get_missing_keys(raw_german_texts):
    # print missing keys
    missing_keys = [ key for key in sorted(all_pagenos) if key not in raw_german_texts or len(raw_german_texts[key]) < 10 ]
    return missing_keys

# Load environment variables from .env file
env_path = Path('../.env')  # Adjust path if needed
load_dotenv(dotenv_path=env_path)

# # Get the root path of the project
sys.path.append(os.path.abspath(".."))

# Display and plotting
from IPython.display import display, HTML, clear_output

# Project imports
from src.utils import timeit, encode_image, plt, pylab
from src.processing import compute_log_spectrum_1d, extract_image_bbox, save_images
from src.api_requests import construct_payload, process_single_page
from src.document_generation import save_document

# Set notebook display width
display(HTML("<style>.container { width:90% !important; }</style>"))

# Print Python environment info
print('sys.executable:', sys.executable)
print('sys.version:', sys.version, '\n')

# Setup for PDF processing
foldername = "Der Weltkrieg v7"

# OpenAI API setup
openai.api_key = os.getenv("OPENAI_API_KEY")
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai.api_key}"
}

# --------------------
# Initialize variables
# --------------------
plotter = False
image_path = f"../input_data/{foldername}/*pdf"
fnames = sorted(glob.glob(image_path))
all_pagenos = [re.search(r'page_(.*?)\.pdf', fname, re.DOTALL).group(1) for fname in fnames]

# Storage for processed texts
raw_german_texts: Dict[str, str] = {}
german_texts: Dict[str, str] = {}
english_texts: Dict[str, str] = {}

# ------------------
# Configure logging
# ------------------
logger = logging.getLogger("time_logger")
if logger.hasHandlers():
    logger.handlers.clear()  # Clear existing handlers to avoid duplicates

logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

# Don't propagate message to parent loggers
logger.propagate = False 


sys.executable: /Users/ozkansafak/code/fraktur/.venv/bin/python3
sys.version: 3.10.9 (main, Mar  1 2023, 12:20:14) [Clang 14.0.6 ] 

CPU times: user 791 ms, sys: 2.49 s, total: 3.28 s
Wall time: 636 ms


---
## Fraktur Translator

In [9]:
@log_execution_time
async def main(fnames, model_name="", semaphore_count=5, extract=True):
    semaphore = asyncio.Semaphore(semaphore_count)  # Adjust number based on API limits
    async def _process_page(fname: str) -> Tuple[str, Dict]:
        global raw_german_texts, german_texts, english_texts  # Explicitly declare globals
        async with semaphore:
            pageno = re.search(r'page_(.*?)\.pdf', fname, re.DOTALL).group(1)
            result = await process_single_page(fname, model_name, headers, plotter, pageno, extract) 
            return pageno, result
    
    # A list of coroutine objects. include only the unprocessed pages.
    keys = set(raw_german_texts.keys())
    tasks = []
    for fname in fnames:
        pageno = re.search(r'page_(.*?)\.pdf', fname, re.DOTALL).group(1)
        if pageno not in keys:
            tasks.append(_process_page(fname))
    logger.info(f"len(tasks): {len(tasks)} -- Process tasks as they complete")
    
    # Process tasks as they complete
    for i, completed_task in enumerate(asyncio.as_completed(tasks)):
        try:
            pageno, (content, raw_german_text, german_text, english_text) = await completed_task
            raw_german_texts[pageno] = raw_german_text
            german_texts[pageno] = german_text
            english_texts[pageno] = english_text
            logger.info(f" {i:3d} of {len(tasks)} -- Successfully processed page:{pageno}")
        except Exception as e:
            logger.error(f"{i:3d} of {len(tasks)} -- Error processing a task: {e}")
    return 

# Run the async code
await main(fnames, "gpt-4o-2024-08-06")


2024-12-20 17:58:10,128 - INFO - len(tasks): 389 -- Process tasks as they complete
2024-12-20 17:58:28,630 - INFO -    0 of 389 -- Successfully processed page:001
2024-12-20 17:58:39,551 - INFO -    1 of 389 -- Successfully processed page:016
2024-12-20 17:58:42,039 - INFO -    2 of 389 -- Successfully processed page:015
2024-12-20 17:58:47,603 - INFO -    3 of 389 -- Successfully processed page:003
2024-12-20 17:58:52,924 - INFO -    4 of 389 -- Successfully processed page:013
2024-12-20 17:58:54,443 - INFO -    5 of 389 -- Successfully processed page:017
2024-12-20 17:59:05,318 - INFO -    6 of 389 -- Successfully processed page:012
2024-12-20 17:59:08,254 - INFO -    7 of 389 -- Successfully processed page:014
2024-12-20 17:59:10,091 - INFO -    8 of 389 -- Successfully processed page:004
2024-12-20 17:59:15,252 - INFO -    9 of 389 -- Successfully processed page:008
2024-12-20 17:59:26,580 - INFO -   10 of 389 -- Successfully processed page:011
2024-12-20 17:59:45,046 - INFO -   11

##  Handle missing keys


In [42]:
missing_keys = set(get_missing_keys(raw_german_texts))
print(f"missing_keys: {missing_keys}")

missing_keys: {'405'}


In [41]:
missing_keys = set(get_missing_keys(raw_german_texts))
print(f"missing_keys: {missing_keys}")
# ---------------------

missed_fnames = []
for fname in fnames:
    pageno = re.search(r'page_(.*?)\.pdf', fname, re.DOTALL).group(1)
    if pageno in missing_keys:
        print(f"processing pageno: {pageno}")
        missed_fnames.append(fname)
        
print(f"missed_fnames: ")
[print(item) for item in missed_fnames]

# Rerun the missing pages without performing FFT based extraction. 

await main(missed_fnames, model_name="claude-3-5-sonnet-20241022", semaphore_count=1, extract=False)


2024-12-20 22:21:01,054 - INFO - len(tasks): 3 -- Process tasks as they complete


missing_keys: {'412', '405', '005', '449'}
processing pageno: 005
processing pageno: 405
processing pageno: 412
processing pageno: 449
missed_fnames: 
../input_data/Der Weltkrieg v7/page_005.pdf
../input_data/Der Weltkrieg v7/page_405.pdf
../input_data/Der Weltkrieg v7/page_412.pdf
../input_data/Der Weltkrieg v7/page_449.pdf


2024-12-20 22:21:24,500 - INFO -    0 of 3 -- Successfully processed page:005
2024-12-20 22:21:36,131 - INFO -    1 of 3 -- Successfully processed page:412
2024-12-20 22:21:49,622 - INFO -    2 of 3 -- Successfully processed page:449
2024-12-20 22:21:49,625 - INFO - Finished main in 48.57 seconds.


In [48]:
# Save json files and .docx files.

from src.document_generation import save_document

# save json outputs
if not os.path.exists(f'../output_data/{foldername}'):
    os.makedirs(f'../output_data/{foldername}')

with open(f'../output_data/{foldername}/english_texts.json', 'w') as f:
    json.dump(english_texts, f)
with open(f'../output_data/{foldername}/german_texts.json', 'w') as f:
    json.dump(german_texts, f)
with open(f'../output_data/{foldername}/raw_german_texts.json', 'w') as f:
    json.dump(raw_german_texts, f)

doc1, fname1 = save_document(german_texts, foldername, language='German')
doc2, fname2 = save_document(english_texts, foldername, language='English')


2024-12-20 22:25:12,109 - INFO - Page 001
2024-12-20 22:25:12,111 - INFO - Page 002
2024-12-20 22:25:12,114 - INFO - Page 003
2024-12-20 22:25:12,116 - INFO - Page 004
2024-12-20 22:25:12,117 - INFO - Page 005
2024-12-20 22:25:12,119 - INFO - Page 006
2024-12-20 22:25:12,121 - INFO - Page 007
2024-12-20 22:25:12,124 - INFO - Page 008
2024-12-20 22:25:12,127 - INFO - Page 009
2024-12-20 22:25:12,130 - INFO - Page 010
2024-12-20 22:25:12,135 - INFO - Page 011
2024-12-20 22:25:12,138 - INFO - Page 012
2024-12-20 22:25:12,140 - INFO - Page 013
2024-12-20 22:25:12,142 - INFO - Page 014
2024-12-20 22:25:12,144 - INFO - Page 015
2024-12-20 22:25:12,145 - INFO - Page 016
2024-12-20 22:25:12,146 - INFO - Page 017
2024-12-20 22:25:12,148 - INFO - Page 018
2024-12-20 22:25:12,150 - INFO - Page 019
2024-12-20 22:25:12,152 - INFO - Page 020
2024-12-20 22:25:12,154 - INFO - Page 021
2024-12-20 22:25:12,156 - INFO - Page 022
2024-12-20 22:25:12,158 - INFO - Page 023
2024-12-20 22:25:12,160 - INFO - P

``` 
1.  Upload Input folder of pdfs to blob storage.
2.  Read file from s3.
3.  FFT in y -> (x_hi, x_lo), write half_cropped_image to s3
4.  FFT in x -> (y_hi, y_lo), write cropped_image to s3
5.  Read cropped image from s3 -> encode_image -> translate and transcribe -> JSON output

```

### Available models and pricing:
```
"gpt-4o-2024-08-06":
    "price_txt": "$2.50 / 1M input tokens"
    "price_img": "$0.001913 / 1500px^2"
    
"gpt-4o-mini-2024-07-18":
    "price_txt": "$0.150 / 1M input tokens"
    "price_img": "$0.003825 / 1500px^2"
    
```

---
## Experiments

In [55]:
base64_image = None
payload = {
    "model": model_name,
    "messages": [
      {"role": "system", "content": "You have three roles. First of all you are a professional OCR assistant. "
       "Secondly, you identify the parts of your transcriptions to belong to header, body and footer sections. "
       "Lastly, you are a GERMAN to ENGLISH translator that stays loyal to the style and "
       "character of the original German text."
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": 
"""Instructions:

You are to perform three steps on the provided image of a document.

**Step 1: OCR Transcription**

Task: Transcribe the entire text from the image into German, including all Fraktur characters.

Attention: Pay close attention to accurately capturing all text elements.

Attention 2: Make sure you're reading each line only once. 

Formatting: Wrap the entire transcription in <raw_german></raw_german> tags.

Caution: Pay attention to identify the paragraphs as a whole and not erroneously place a carriage return at the end of each line.

Separator: When you are done with Step 1, print the separator line:

--------------------------------------------------------------------
**Step 2: Header-Body-Footer Analysis**

Review: Look at the image and your transcription from Step 1.

Verification: Ensure you haven't missed any parts; if you did, transcribe and include them now.

Caution: Pay attention to identify the paragraphs as a whole and not erroneously place a carriage return at the end of each line.

Categorization:

Header: If you detect a header (e.g., chapter title or section heading), wrap it inside <header></header> tags. If there's no header, omit the <header></header> tags.
Body: Wrap the main body of the text inside <body></body> tags.
Footer: If you detect any footnotes, wrap them inside <footer></footer> tags. If there are no footnotes, omit the <footer></footer> tags.
Formatting: Wrap this structured transcription inside <german></german> tags.

Separator: When you are done with Step 2, print the separator line again:

--------------------------------------------------------------------
**Step 3: Translation (German to English)**

Task: Translate the structured German text from Step 2 into English.
Structure: Maintain the same <header>, <body>, and <footer> sections in your translation.
Formatting: Wrap the translated text inside <english></english> tags.

Example Output Format:

<raw_german>
... (transcribed German text) ...
</raw_german>
--------------------------------------------------------------------
<german>
<header> ... </header>
<body> ... </body>
<footer> ... </footer>
</german>
--------------------------------------------------------------------
<english>
<header> ... </header>
<body> ... </body>
<footer> ... </footer>
</english>"""                

          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    "max_tokens": 6000,
    "temperature": 0.1
}


In [54]:
payload.keys()


dict_keys(['model', 'messages', 'max_tokens', 'temperature'])

In [78]:
payload['messages'][1]['content'][1]

{'type': 'image_url', 'image_url': {'url': 'data:image/jpeg;base64,None'}}

In [102]:
payload['messages'][1]['content'][1]

{'type': 'image_url', 'image_url': {'url': 'data:image/jpeg;base64,None'}}